In [3]:
print("====================")
import pandas as pd 
import os

import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa

import tldextract
import urllib

import sys
import re

import sys
sys.path.append("/scratch/venia/web2wiki/code/helpers/")

from settings import DATA_DIR

print("====================")

regex_subset = "(\#[\/a-zA-ZäöüÄÖÜßþóúí\_\(\),\,\-\#\&\$\@\!0-9\.\%\–\'\:]+)"
regex_final = "([^#]*)"

os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"
spark = SparkSession.builder.getOrCreate()


22/09/26 22:18:05 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/09/26 22:18:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/26 22:18:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/26 22:18:05 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/09/26 22:18:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/26 22:18:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [24]:
import glob
files = glob.glob("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/tag_info_10_*")
df = spark.read.load(files)

In [25]:
df.printSchema()

root
 |-- url: string (nullable = true)
 |-- wiki_links: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_tag_footer: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_header: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_aside: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_nav: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_sup: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_cite: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_p: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_tag_article: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_class_footer: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- is_class_header: array (nullable = true)

In [27]:
df = df.dropDuplicates(subset = ["url"])
df = df.withColumn("new", F.arrays_zip("wiki_links", "is_tag_footer","is_tag_header","is_tag_aside","is_tag_sup","is_tag_cite","is_tag_p","is_tag_article","is_class_footer","is_class_header","is_class_sidebar","is_class_comment","previous_header","nbhd_text"))\
       .withColumn("new", F.explode("new"))\
       .select("url", F.col("new.wiki_links").alias("wiki_links"), F.col("new.is_tag_footer").alias("is_tag_footer"),F.col("new.is_tag_header").alias("is_tag_header"),F.col("new.is_tag_aside").alias("is_tag_aside"), F.col("new.is_tag_sup").alias("is_tag_sup"),F.col("new.is_tag_cite").alias("is_tag_cite"),F.col("new.is_tag_p").alias("is_tag_p"), F.col("new.is_tag_article").alias("is_tag_article"), F.col("new.is_class_footer").alias("is_class_footer"),F.col("new.is_class_header").alias("is_class_header"),F.col("new.is_class_sidebar").alias("is_class_sidebar"), F.col("new.is_class_comment").alias("is_class_comment"), F.col("new.previous_header").alias("previous_header"), F.col("new.nbhd_text").alias("nbhd_text"))
df = df.toPandas()

In [ ]:
df2 = df


In [21]:
df2

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_nav,is_tag_sup,is_tag_cite,is_class_footer,is_class_header,is_class_sidebar,is_class_comment
0,http://anewsreporter.weebly.com/anewsreporterc...,http://en.wikipedia.org/wiki/Myron_Coureval_Fagan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,http://anewsreporter.weebly.com/anewsreporterc...,https://en.wikipedia.org/wiki/GcMAF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,http://anewsreporter.weebly.com/anewsreporterc...,http://en.wikipedia.org/wiki/Anticaking_agent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,http://anewsreporter.weebly.com/anewsreporterc...,http://en.wikipedia.org/wiki/High-availability...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,http://anewsreporter.weebly.com/anewsreporterc...,https://en.wikipedia.org/wiki/Johnson_Amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1613333,http://www.universetoday.com/147760/vera-rubin...,https://www6.slac.stanford.edu/news/2020-09-08...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1613334,http://www.utahpoliticohub.com/heres-why-judge...,http://en.wikipedia.org/wiki/Anthony_Quinn_(ju...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1613335,http://www.utahpoliticohub.com/heres-why-judge...,https://www.patreon.com/utahpoliticohub?utm_co...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1613336,http://www.wordaz.com/Looting.html,http://en.wikipedia.org/wiki/Looting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
df = pd.read_csv("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/cleaner_data.csv")

In [111]:
df = df[df["link_type"] == 1]

In [112]:
df.groupby("order")["order"].count()

order
0     50272
1    620447
2     18561
Name: order, dtype: int64

In [114]:
df[df["order"] == 1]["first_class"].unique()

array(['ws', 'attribution', 'evidence'], dtype=object)

In [ ]:
df = df.dropna()

In [ ]:
df = df.fillna(0)

In [61]:
df["is_class_sidebar"].sum()

45223.0

In [75]:
order_0 = ["is_tag_footer", "is_tag_header","is_tag_aside","is_class_footer","is_class_header","is_class_sidebar"]


In [ ]:
df = pd.read_csv("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/clean_data.csv")
df = df.drop_duplicates(subset = ["url","wiki_links"])
df = df.fillna(0)
def try_float(x):
    try:
        float(x)
        return True
    except:
        return False


df["previous_header"] = df["previous_header"].astype(str)
df["is_reference"] = df["previous_header"].apply(lambda x: 1 if "reference" in x.lower() else 0)

In [94]:

order_0 = ["is_tag_footer", "is_tag_header","is_tag_aside","is_class_footer","is_class_header","is_class_sidebar"]
for col in order_0:
    df = df[df[col].map(try_float)]

evidence = ["is_tag_sup","is_tag_cite","is_reference"]
for col in evidence:
    df = df[df[col].map(try_float)]
order_2 = ["is_class_comment"]
for col in order_2:
    df = df[df[col].map(try_float)]

In [93]:
df[df["is_tag_footer"].map(try_float)]

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,is_class_header,is_class_sidebar,is_class_comment,previous_header,nbhd_text,is_wiki,url_wiki_count,is_blog,is_reference
0,http://09130.mywheels.dk/faelge/bil/FORD/GT,http://en.wikipedia.org/wiki/HTTP_cookie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,\ni\nDet her site bruger [BREAK] cookies [BRE...,0.0,2.0,0.0,0
1,http://09130.mywheels.dk/faelge/bil/FORD/GT,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,None,[BREAK] \n\n [BREAK] \n\n\n\n\n\n\n\nShopfors...,0.0,2.0,0.0,0
2,http://123moviescom.com/movie/watch-the-town-2...,https://en.wikipedia.org/wiki/The_Town_(2010_f...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,[BREAK] [BREAK] \n\n\n\n,0.0,2.0,0.0,0
3,http://123moviescom.com/movie/watch-the-town-2...,/,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,\n\n\n [BREAK] All Movies [BREAK] Watch The ...,0.0,2.0,0.0,0
4,http://19trandafiri.wordpress.com/tag/schizofr...,http://en.wikipedia.org/wiki/Dorothy_Otnow_Lewis,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,„ [BREAK] Dorothy Otnow Lewis [BREAK] a absol...,0.0,10.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18868679,http://zims-en.kiwix.campusafrica.gos.orange.c...,https://en.wikipedia.org/wiki/Schulze_method?o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,\n This article is issued from\n [BREAK...,1.0,2.0,0.0,0
18868680,http://zims-en.kiwix.campusafrica.gos.orange.c...,Sortition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,\n This article is issued from\n \n ...,1.0,2.0,0.0,0
18868681,http://zxspectrum4.net/cassette_tapes.php,http://en.wikipedia.org/wiki/ZX_Spectrum_software,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,\n\t\t\t How does it wor...,0.0,4.0,0.0,0
18868682,http://zxspectrum4.net/cassette_tapes.php,http://en.wikipedia.org/wiki/MD5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,ve the bytes which exist on the tape using thi...,0.0,4.0,0.0,0


In [105]:
a = df[order_0].astype(float).sum(axis = 1)

In [106]:
a.sum()

216410.0

In [82]:
df["is_class_sidebar"].sum()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [70]:
t = df[df["is_class_sidebar"] != 0]

In [72]:
t[order_0].sum(axis =1)

/tmp/ipykernel_1228277/3064387606.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  t[order_0].sum(axis =1).sum()


45281.0

In [64]:
counts = df[order_0].sum(axis = 1)

/tmp/ipykernel_1228277/2910107466.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  counts = df[order_0].sum(axis = 1)


In [65]:
counts.sum()

9474.0

In [66]:
counts

0           0.0
2           0.0
3           0.0
4           0.0
5           0.0
           ... 
18868680    0.0
18868681    0.0
18868682    0.0
18868683    0.0
18868684    0.0
Length: 1584294, dtype: float64

In [54]:
df[df["link_type"] == 1].groupby("first_class").count()

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,...,is_blog,is_reference,0th_order,evidence,attribution,2nd_order,order,link_type,domain,domain_count
first_class,,,,,,,,,,,,,,,,,,,,,
attribution,9884,9884,9884,9884,9884,9884,9884,9884,9884,9884,...,9884,9884,9884,9884,9884,9884,9884,9884,9884,9884
evidence,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,...,6693,6693,6693,6693,6693,6693,6693,6693,6692,6693
ws,650624,650624,650624,650624,650624,650624,650624,650624,650624,650624,...,650624,650624,650624,650624,650624,650624,650624,650624,650586,650624


In [56]:
df[df["link_type"] == 1].groupby("order").sum()

,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,is_class_header,is_class_sidebar,is_class_comment,is_wiki,url_wiki_count,is_blog,is_reference,0th_order,evidence,2nd_order,link_type,domain_count
order,,,,,,,,,,,,,,,,,,,,
0,1181.0,127.0,16.0,2.0,3.0,662.0,88.0,2657.0,555.0,9.0,29.0,585.0,67963.0,154.0,0,3212.0,0,29.0,3212,2508037
1,10223.0,5854.0,17065.0,8915.0,1698.0,348565.0,228383.0,0.0,0.0,23329.0,0.0,94111.0,137039036.0,188354.0,1399,0.0,1399,0.0,667201,606098279
2,152.0,86.0,73.0,20.0,1.0,17534.0,10197.0,0.0,0.0,2.0,18867.0,187.0,426808.0,12687.0,0,0.0,0,18867.0,18867,7730218


In [28]:
cols = list(df.columns[1:])


In [30]:
for col in cols[1:]:
    df[col] = df[col].astype(int)
df["is_wiki"] = df["url"].apply(lambda x: 1 if "wiki" in x or "peida" in x else 0)


In [38]:
pattern = re.compile(r"[12][90][89012][0123456789]/[01]\d")
df["is_blog"] = df["url"].apply(lambda x: 1 if "blog" in x or bool(pattern.search(x)) else 0)
df["url_wiki_count"]=df.groupby("url")["is_tag_footer"].transform("count")
df = df.drop_duplicates(subset = "url")


In [39]:
df.sample(1200).to_csv("/scratch/venia/web2wiki/data/tag_info_clean2.csv",index=False)

In [50]:
df.columns

Index(['url', 'wiki_links', 'is_tag_footer', 'is_tag_header', 'is_tag_aside',
       'is_tag_sup', 'is_tag_cite', 'is_tag_p', 'is_tag_article',
       'is_class_footer', 'is_class_header', 'is_class_sidebar',
       'is_class_comment', 'previous_header', 'nbhd_text', 'is_wiki',
       'url_wiki_count', 'is_blog', 'is_reference', '0th_order', 'evidence',
       'attribution', '2nd_order', 'order', 'first_class', 'link_type',
       'domain', 'domain_count'],
      dtype='object')

In [51]:
df[df["0th_order"]>0]

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,...,is_reference,0th_order,evidence,attribution,2nd_order,order,first_class,link_type,domain,domain_count
263,http://al-quran.info/74/5/Tb,https://en.wikipedia.org/wiki/Quran,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,1,al-quran.info,507
404,http://ancienthebrewpoetry.typepad.com/ancient...,https://ancienthebrewpoetry.typepad.com/ancien...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,0,ancienthebrewpoetry.typepad.com,4
550,http://api.aprs.fi/info/a/DO1LMD,https://aprs.fi/doc/guide/guide.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,0,api.aprs.fi,223
1325,http://brandsprice.com/coach%20%E8%8D%94%E6%9E...,https://en.wikipedia.org/wiki/E-commerce,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,1,brandsprice.com,22
1702,http://citycomfortsblog.typepad.com/cities,https://citycomfortsblog.typepad.com/cities/20...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1.0,0,0,0.0,0,NaN,0,citycomfortsblog.typepad.com,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994181,http://www.peoplefinders.com/name/yazmin-pelae...,https://en.wikipedia.org/wiki/Fair_Credit_Repo...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,1,www.peoplefinders.com,256
995991,http://www.sindome.org/wiki/index.php?oldid=68...,https://www.sindome.org/wiki/index.php?title=W...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,0,www.sindome.org,42
995995,http://www.sixthinline.com/have-you-ever-been-...,https://www.sixthinline.com/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1.0,0,0,0.0,0,NaN,0,www.sixthinline.com,18
996716,http://www.wonderzine.com.ua/tags/%D0%9A%D1%96...,https://en.wikipedia.org/wiki/HTTP_cookie,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0,1.0,0,0,0.0,0,NaN,1,www.wonderzine.com.ua,41


In [ ]:
df[df["link_type"] ==1]

In [49]:
df[df["url_wiki_count"]< 10].groupby("order").agg("count")

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,...,is_blog,is_reference,0th_order,evidence,attribution,2nd_order,first_class,link_type,domain,domain_count
order,,,,,,,,,,,,,,,,,,,,,
0,4321,4321,4321,4321,4321,4321,4321,4321,4321,4321,...,4321,4321,4321,4321,4321,4321,0,4321,4254,4321
1,442336,442336,442336,442336,442336,442336,442336,442336,442336,442336,...,442336,442336,442336,442336,442336,442336,442336,442336,442277,442336
2,21745,21745,21745,21745,21745,21745,21745,21745,21745,21745,...,21745,21745,21745,21745,21745,21745,0,21745,21745,21745


In [ ]:
df.groupby("order").count()